In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


# Read data

In [204]:
train = pd.read_csv('./msbd5001-spring-2022/train.csv',index_col=0)
test = pd.read_csv('./msbd5001-spring-2022/test.csv',index_col=0)

Check to see if there's any null values

In [205]:
train.isnull().sum()

MO HLADR+ MFI (cells/ul)    1
Neu CD64+MFI (cells/ul)     1
CD3+T (cells/ul)            0
CD8+T (cells/ul)            0
CD4+T (cells/ul)            0
NK (cells/ul)               0
CD19+ (cells/ul)            0
CD45+ (cells/ul)            0
Age                         0
Sex 0M1F                    0
Mono CD64+MFI (cells/ul)    1
label                       0
dtype: int64

There are null values but luckily we are just loosing 1 row of entry.

In [206]:
train.dropna(inplace=True)
train.describe()

,MO HLADR+ MFI (cells/ul),Neu CD64+MFI (cells/ul),CD3+T (cells/ul),CD8+T (cells/ul),CD4+T (cells/ul),NK (cells/ul),CD19+ (cells/ul),CD45+ (cells/ul),Age,Sex 0M1F,Mono CD64+MFI (cells/ul),label
count,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000
mean,1264.244186,290.383721,978.454186,476.313372,494.259884,214.410465,117.928721,1321.717558,40.441860,0.488372,2066.534884,0.337209
std,765.452376,490.283499,619.751401,345.179243,313.607350,173.860206,96.451856,795.614103,10.312305,0.502797,1198.401364,0.475530
min,112.000000,30.000000,74.400000,36.610000,39.590000,0.000000,4.200000,209.250000,19.000000,0.000000,72.000000,0.000000
25%,685.500000,77.500000,548.005000,236.880000,270.977500,81.730000,52.182500,778.807500,33.250000,0.000000,1461.250000,0.000000
50%,1108.500000,124.500000,865.095000,420.850000,447.115000,191.235000,88.410000,1179.270000,41.000000,0.000000,1757.500000,0.000000
75%,1602.250000,244.500000,1226.370000,620.815000,624.830000,264.862500,151.972500,1608.147500,49.750000,1.000000,2238.250000,1.000000
max,4145.000000,3124.000000,3791.230000,2548.100000,1517.810000,878.040000,485.860000,4757.280000,60.000000,1.000000,7515.000000,1.000000


Separate predictors and label

In [207]:
X = train.drop('label',axis=1) #Predictors
y = train['label'] #Label
X.head()


,MO HLADR+ MFI (cells/ul),Neu CD64+MFI (cells/ul),CD3+T (cells/ul),CD8+T (cells/ul),CD4+T (cells/ul),NK (cells/ul),CD19+ (cells/ul),CD45+ (cells/ul),Age,Sex 0M1F,Mono CD64+MFI (cells/ul)
id,,,,,,,,,,,
0,3556.0,2489.0,265.19,77.53,176.55,0.00,4.20,307.91,52,0,7515.0
1,1906.0,134.0,1442.61,551.90,876.07,112.10,168.15,1735.48,20,1,1756.0
2,1586.0,71.0,1332.74,684.20,655.26,244.95,216.52,1820.04,28,1,1311.0
3,683.0,94.0,419.23,255.80,162.17,72.05,44.68,538.22,55,1,1443.0
4,1032.0,71.0,1102.72,480.27,625.30,188.78,130.77,1427.97,28,1,1542.0


# Split training and validation set

In [262]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Models

## Logistic regression

In [133]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0,solver='lbfgs', max_iter=1000).fit(X_train, y_train)
clf.score(X_test,y_test)


0.8888888888888888

## Random Forest

In [263]:
from sklearn.ensemble import RandomForestClassifier

clf2 = RandomForestClassifier(max_depth=20,n_estimators = 17).fit(X_train, y_train)
clf2.score(X_test,y_test)

0.9444444444444444

In [210]:
label_pred2 = clf2.predict(test)

In [212]:
# Generate submission:
submission2 = pd.DataFrame({'id':np.arange(len(label_pred2)),'label': label_pred2.astype(int)})
# submission.head()
submission2.to_csv('submission2.csv', index=False)

## Adaboost

In [138]:
from sklearn.ensemble import AdaBoostClassifier
clf3 = AdaBoostClassifier(n_estimators = 50).fit(X_train, y_train)
clf3.score(X_test,y_test)

0.8333333333333334

## SVM

In [139]:
from sklearn import svm
clf4 = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
clf4.score(X_test,y_test)

0.8888888888888888

In [143]:
clf4 = svm.SVC(kernel='poly', C=1).fit(X_train, y_train)
clf4.score(X_test,y_test)

0.9444444444444444

In [141]:
clf5 = svm.SVC(kernel='rbf', C=1).fit(X_train, y_train)
clf5.score(X_test,y_test)

0.7222222222222222

## KNN

In [149]:
from sklearn.neighbors import KNeighborsClassifier
clf6 = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)
clf6.score(X_test,y_test)

0.9444444444444444

## Decision Tree

In [151]:
from sklearn import tree
clf7 = tree.DecisionTreeClassifier(criterion="gini").fit(X_train, y_train)
clf7.score(X_test,y_test)

0.9444444444444444

## Neural Network

In [152]:
import torch
import torch.optim as optim
from torch import nn
import torch.nn.functional as F

In [153]:
# Randomize initial weights 
def init_weights_u(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight,std=0.01)

class BinaryClassNet(nn.Module):
    def __init__(self,input_units,hidden_units):
        super(BinaryClassNet,self).__init__()
        self.fc1 = nn.Linear(input_units, hidden_units)
        self.fc2 = nn.Linear(hidden_units,1)
        self.apply(init_weights_u)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

In [162]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [155]:
import torch
import torch.optim as optim
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
import time
from IPython.display import display


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.model_selection import StratifiedKFold

In [223]:

skf = StratifiedKFold(n_splits=3,random_state=42,shuffle=True)
trainset_split = skf.split(X,y)
train_idx, test_idx = next(trainset_split.__iter__())

# Set batch size
trainset_train = TensorDataset(torch.Tensor(np.array(X.iloc[train_idx,])),torch.Tensor(np.array(y.iloc[train_idx]))) 
trainset_train_loader = DataLoader(trainset_train,batch_size=32) 

In [224]:
net = BinaryClassNet(11,5)

criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

In [228]:
losses = []
for epoch in range(100):  # loop over the dataset multiple times

    running_loss = 0.0
    
    for i, data in enumerate(trainset_train_loader, 0):
        
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = net(inputs)
        loss = criterion(outputs, labels.reshape(-1,1))

        #accuracy
        predicted_test = net(torch.tensor(np.array(X.iloc[test_idx,]),dtype=torch.float32))
        acc_test = (predicted_test.reshape(-1).detach().numpy().round() == np.array(y.iloc[test_idx])).mean()
        predicted_train = net(torch.tensor(np.array(X.iloc[train_idx,]),dtype=torch.float32))
        acc_train = (predicted_train.reshape(-1).detach().numpy().round() == np.array(y.iloc[train_idx])).mean()

        #backprop
        loss.backward()
        optimizer.step()

        # print statistics
        if epoch%5 == 0 & i%4==0:
            losses.append(loss)
            #accur.append(acc)
            print(f"epoch: {epoch}\tbatch: {i}\tloss: {loss:.5f}\tacc_train: {acc_train:.5f}\tacc_test: {acc_test:.5f}")

        '''# print statistics
        running_loss += loss.item()
        if i % 2 == 1:
            ###
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2))
            running_loss = 0.0'''

print('Finished Training')

epoch: 0	batch: 0	loss: 0.15519	acc_train: 0.94737	acc_test: 0.96552
epoch: 0	batch: 1	loss: 0.11291	acc_train: 0.92982	acc_test: 0.96552
epoch: 5	batch: 0	loss: 0.15184	acc_train: 0.94737	acc_test: 0.96552
epoch: 5	batch: 1	loss: 0.11390	acc_train: 0.94737	acc_test: 0.93103
epoch: 10	batch: 0	loss: 0.14888	acc_train: 0.94737	acc_test: 0.96552
epoch: 10	batch: 1	loss: 0.10407	acc_train: 0.94737	acc_test: 0.96552
epoch: 15	batch: 0	loss: 0.14844	acc_train: 0.94737	acc_test: 0.93103
epoch: 15	batch: 1	loss: 0.10221	acc_train: 0.94737	acc_test: 0.96552
epoch: 20	batch: 0	loss: 0.14290	acc_train: 0.94737	acc_test: 0.93103
epoch: 20	batch: 1	loss: 0.09493	acc_train: 0.94737	acc_test: 0.96552
epoch: 25	batch: 0	loss: 0.14180	acc_train: 0.94737	acc_test: 0.96552
epoch: 25	batch: 1	loss: 0.09814	acc_train: 0.94737	acc_test: 0.96552
epoch: 30	batch: 0	loss: 0.35331	acc_train: 0.89474	acc_test: 0.82759
epoch: 30	batch: 1	loss: 0.13643	acc_train: 0.94737	acc_test: 0.96552
epoch: 35	batch: 0	loss:

In [229]:
PATH = './pneumonia.pth'
torch.save(net.state_dict(), PATH)

In [230]:
test_set_pred = net(torch.tensor(np.array(test),dtype=torch.float32))

In [231]:
label_pred = test_set_pred.reshape(-1).detach().numpy().round()

In [232]:
# Generate submission:
submission = pd.DataFrame({'id':np.arange(len(label_pred)),'label': label_pred.astype(int)})
# submission.head()
submission.to_csv('submission_1.csv', index=False)